##  Calculate Yearly DPLE variables

In [1]:
import xarray as xr
import numpy as np
import metabolic as mi
import os
import pandas as pd
import utils
import warnings
warnings.filterwarnings('ignore')

In [2]:
USER = os.environ['USER']
a1, a2, b1, b2 = [240, 380, 125, 320]

In [10]:
var = 'TEMP'
#var = 'O2'
#var = 'AOU'
#var = 'SALT'

In [11]:
path = f'/glade/scratch/{USER}/DPLE-results'
filename = f'{path}/DPLE_driftcorrected_{var}_ens_mean.nc'
ds0 = xr.open_dataset(filename)
ds0[var]

<xarray.DataArray 'SALT' (Y: 64, L: 122, z_t: 60, nlat: 384, nlon: 320)>
[57566822400 values with dtype=float32]
Coordinates:
  * z_t      (z_t) float32 500.0 1.5e+03 2.5e+03 ... 5.125e+05 5.375e+05
    TLAT     (nlat, nlon) float64 ...
    ULONG    (nlat, nlon) float64 ...
    ULAT     (nlat, nlon) float64 ...
    TLONG    (nlat, nlon) float64 ...
  * Y        (Y) int64 1954 1955 1956 1957 1958 ... 2013 2014 2015 2016 2017
  * L        (L) int64 1 2 3 4 5 6 7 8 9 ... 114 115 116 117 118 119 120 121 122
Dimensions without coordinates: nlat, nlon

In [12]:
filename = f'{path}/DPLE_driftcorrected_{var}_mean_ens_mean.nc'
ds1 = xr.open_dataset(filename)
ds1

<xarray.Dataset>
Dimensions:  (z_t: 60, nlat: 384, nlon: 320, Y: 64, L: 122)
Coordinates:
  * z_t      (z_t) float32 500.0 1.5e+03 2.5e+03 ... 5.125e+05 5.375e+05
    TLAT     (nlat, nlon) float64 ...
    ULONG    (nlat, nlon) float64 ...
    ULAT     (nlat, nlon) float64 ...
    TLONG    (nlat, nlon) float64 ...
  * Y        (Y) int64 1954 1955 1956 1957 1958 ... 2013 2014 2015 2016 2017
  * L        (L) int64 1 2 3 4 5 6 7 8 9 ... 114 115 116 117 118 119 120 121 122
Dimensions without coordinates: nlat, nlon
Data variables:
    SALT     (Y, L, z_t, nlat, nlon) float64 ...

In [13]:
dvar = ds0[var].isel(z_t=range(0,35),nlat=range(a1,a2),nlon=range(b1,b2)) - ds1[f'{var}'].isel(z_t=range(0,35),nlat=range(a1,a2),nlon=range(b1,b2))

In [14]:
dsf = xr.Dataset()
dsf[f'd{var}'] = dvar

In [15]:
dsf.load()

<xarray.Dataset>
Dimensions:  (z_t: 35, nlat: 140, nlon: 195, Y: 64, L: 122)
Coordinates:
  * z_t      (z_t) float32 500.0 1.5e+03 2.5e+03 ... 5.277e+04 5.794e+04
    TLAT     (nlat, nlon) float64 14.96 14.96 14.97 14.97 ... 70.43 70.42 70.42
    ULONG    (nlat, nlon) float64 101.7 102.9 104.0 105.1 ... 318.9 319.5 320.0
    ULAT     (nlat, nlon) float64 15.13 15.13 15.14 15.14 ... 70.64 70.63 70.63
    TLONG    (nlat, nlon) float64 101.2 102.3 103.4 104.5 ... 318.6 319.2 319.7
  * Y        (Y) int64 1954 1955 1956 1957 1958 ... 2013 2014 2015 2016 2017
  * L        (L) int64 1 2 3 4 5 6 7 8 9 ... 114 115 116 117 118 119 120 121 122
Dimensions without coordinates: nlat, nlon
Data variables:
    dSALT    (Y, L, z_t, nlat, nlon) float64 nan nan nan nan ... nan nan nan nan

In [16]:
dout = f'/glade/scratch/{USER}/DPLE-results'
os.makedirs(dout, exist_ok=True)
dsf.to_netcdf(f'{dout}/DPLE-driftcorrected_d{var}_ens_mean_monthly.nc', mode='w')